# Interrupted Time Series Analysis - Data Pre-Processing

Basic steps in pre-processing
1. Read in all ED attendences (adult and children)
2. Filter for age >= 18
3. Create flags for arrivals 0600-2200 (day) and 2200-0600 (night)
4. Filter for day of week = Mon-Thur
5. Create monthly aggregates for outcomes and explanatory variables (e.g mean_total_time)
6. Create time series specification dataset (e.g. add level, trend variables)
7. Save dataset to file.

## Step 1: Read in all data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:

col_names = ['atd_no','age','arrival_time','arrival_mode','clock_stop','bed_request','speciality_ref','total_time',
             'flag_breach','flag_admit','flag_reatten']

df = pd.read_csv('./Attendances.txt',names=col_names)

In [ ]:
df.head(5)

In [ ]:
df.shape

## Step 2: Filter by Age

In [ ]:
df = df.loc[df['age']>=18]


In [ ]:
df['on_target'] = np.where(df['flag_breach'] == 1, 0, 1)

In [ ]:
df['arrival_time_d'] = pd.to_datetime(df['arrival_time'], dayfirst = True)

In [ ]:
df['arrival_hr']= df['arrival_time_d'].dt.hour




In [ ]:
df.head(5)

## Step 3: Flag for night and day

In [ ]:
df['night_att'] = np.where(np.logical_or(df['arrival_hr'] >= 22, df['arrival_hr'] <= 6), 1, 0)

## Step 4: Flag for Mon-Thu

In [ ]:
df['dow'] = df['arrival_time_d'].dt.dayofweek

Notes for dt.dayofweek. Monday = 0 and Sunday = 6

In [ ]:
df['dow_mon_thur'] = np.where(df['dow'] <=3, 1, 0)

In [ ]:
df['year_mth'] = pd.DatetimeIndex(df['arrival_time_d']).normalize()

In [ ]:
df['arrival_time_d'].dt.month.head(4)

In [ ]:
df['year_mth_only'] = df['arrival_time_d'].values.astype('datetime64[M]')

In [ ]:
df.head(5)

In [ ]:
df = df.loc[df['age']>18]

In [ ]:
df.shape

## Create monthly aggregates

In [ ]:
month_series_mean = df.groupby(['year_mth_only', 'night_att', 'dow_mon_thur'])['total_time'].mean()
month_series_mean.rename('mean_total_time', inplace=True).head()

In [ ]:
month_series_n = df.groupby(['year_mth_only', 'night_att', 'dow_mon_thur'])['total_time'].count()
month_series_n.rename('patients_n', inplace=True).head()

In [ ]:
month_series_target = df.groupby(['year_mth_only', 'night_att', 'dow_mon_thur'])['on_target'].sum()
month_series_target.rename('on_target', inplace=True).head()

In [ ]:
month_series_admit = df.groupby(['year_mth_only', 'night_att', 'dow_mon_thur'])['flag_admit'].sum()
month_series_admit.rename('admit_n', inplace=True).head()

In [ ]:
df_month = pd.concat([month_series_mean, month_series_n, month_series_target, month_series_admit], axis=1)
df_month['per_on_target'] = df_month['on_target'] / df_month['patients_n']
df_month['per_admit'] = df_month['admit_n'] / df_month['patients_n']

In [ ]:
df_month.reset_index(inplace=True)


In [ ]:
df_month['level'] = np.where(df_month['year_mth_only']>='2015-11-01', 1, 0)

In [ ]:
df_month.head()

In [ ]:
#limit to Monday to Friday
df_week = df_month.loc[df_month['dow_mon_thur'] == 1]
df_week.head(5)

In [ ]:
#limit to Night Performance
df_nights = df_week.loc[df_month['night_att'] == 1]
df_nights.reset_index(inplace=True)

In [ ]:
df_nights['time'] = df_nights.index + 1
df_nights

In [ ]:
df_nights['trend'] = np.where(df_nights['year_mth_only']>='2015-11-01',df_nights.index - 33 , 0)
df_nights['group'] = 1

In [ ]:
df_nights

In [ ]:
#limit to Day Performance
df_days = df_week.loc[df_month['night_att'] == 0]
df_days.reset_index(inplace=True)

## Time series format for regression

In [ ]:
df_days['time'] = df_days.index + 1
df_days

In [ ]:
df_days['trend'] = np.where(df_days['year_mth_only']>='2015-11-01',df_days.index - 33 , 0)
df_days['group'] = 0

In [ ]:
df_days

In [ ]:
df_days.drop(labels = 'index', inplace=True, axis=1)
df_nights.drop(labels = 'index', inplace=True, axis=1)

In [ ]:
df_ts_spec = pd.concat([df_nights, df_days])

In [ ]:
df_ts_spec['group_time'] = df_ts_spec['group'] * df_ts_spec['time']
df_ts_spec['group_level'] = df_ts_spec['group'] * df_ts_spec['level']
df_ts_spec['group_trend'] = df_ts_spec['group'] * df_ts_spec['trend']


In [ ]:
df_ts_spec.shape

In [ ]:
df_ts_spec

In [ ]:
df_ts_spec.to_csv("20180125_night_day_data_ts.csv")